In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

train = pd.read_csv("D:\\Project\\NLP_Disaster\\data\\train.csv")
test = pd.read_csv("D:\\Project\\NLP_Disaster\\data\\test.csv")

In [11]:
n_train = len(train.index)
positive = len(train.query('target == 1'))
negative = len(train.query('target == 0'))
print(n_train, positive, negative)

7613 3271 4342


In [14]:
sentences_length = train['text'].str.len()
print(sentences_length)

0        69
1        38
2       133
3        65
4        88
       ... 
7608     83
7609    125
7610     65
7611    137
7612     94
Name: text, Length: 7613, dtype: int64
